In [1]:
import os
import numpy as np
import pandas as pd
import getpass
import cPickle as pickle
import pylab as plt
%matplotlib inline
from random import shuffle
import GPy

datasets = '/home/'+ getpass.getuser() +'/Datasets/'
print datasets

file_name = 'tasks_by_skills.p'
p = os.path.join(datasets, 'FoEmployment/Analysis_of_ONET_Tasks', file_name)

with open(p, 'r') as f:
    data = pickle.load(f)
(X, test, y) = data

#Misc settings
fs = (9, 5)  # figure size; change this if you need to
ms = 11  # marker size in plots
sd = 6   # random seed

/home/scpd/Datasets/


In [2]:
# Validate :) 
# from sklearn.metrics import mean_squared_error
# print "validate RMS = ", math.sqrt(mean_squared_error(y_valid, y_pred))   # smaller better :)

def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [3]:
def learn_a_GP(x_train, y_train):
    in_dims = x_train.shape[1]
    
    k = GPy.kern.RBF(in_dims, variance=1, lengthscale=0.4)
    # k = GPy.kern.Matern32(1)
    # k = GPy.kern.Matern52(1)
    # k = GPy.kern.Exponential(1)
    # k = GPy.kern.PeriodicExponential(1)

    m = GPy.models.GPRegression(x_train, y_train, k)
    m.optimize()
    return m

In [4]:
val_size = 0.1
cols = test.columns[2:]
in_dims = cols.shape[0]

# Create a Validation set
msk = [i for i in range(X.shape[0])]

RMS_reps = []
for rep in xrange(10):
    print "repeat:", rep, 
    shuffle(msk)

    n_validation_set = int(np.floor(X.shape[0]*val_size))
#     print msk[:n_validation_set]

    x_train = X.iloc[msk[n_validation_set:]][cols].as_matrix()
    x_valid = X.iloc[msk[:n_validation_set]][cols].as_matrix()
    
    y_train = y[msk[n_validation_set:]].as_matrix().reshape(x_train.shape[0], 1)
    y_valid = y[msk[:n_validation_set]].as_matrix().reshape(n_validation_set, 1)

    x_test = test[cols].as_matrix()

    m = learn_a_GP(x_train, y_train)
    y_pred, Vp = m.predict(x_valid, full_cov=False)
    err = rmse(y_valid, y_pred)
    RMS_reps.append(err)
    print "RMS Error is: " + str(err)
    
print "\nAverage RMS = ", repr(np.mean(RMS_reps))

repeat: 0 RMS Error is: 0.55781393473
repeat: 1 RMS Error is: 0.457754005708
repeat: 2 RMS Error is: 0.617450258
repeat: 3 RMS Error is: 0.549611756616
repeat: 4 RMS Error is: 0.537547745984
repeat: 5 RMS Error is: 0.571268672402
repeat: 6 RMS Error is: 0.565258744882
repeat: 7 RMS Error is: 0.490054401796
repeat: 8 RMS Error is: 0.632487402234
repeat: 9 RMS Error is: 0.555715987136

Average RMS =  0.55349629094889141


In [5]:
print X.shape
print x_train.shape
print y_train.shape
print x_valid.shape
print y_valid.shape
print x_test.shape
print in_dims

(314, 39)
(283, 35)
(283, 1)
(31, 35)
(31, 1)
(1753, 35)
35
